In [1]:
#pip install bs4

In [2]:
#pip install lxml

In [3]:
#pip install requests

In [4]:
#pip install selenium

In [5]:
import csv
from bs4 import BeautifulSoup

In [6]:
from selenium import webdriver

In [7]:
#pip install webdriver-manager

In [8]:
driver = webdriver.Chrome()

In [9]:
url ="https://www.amazon.com"
driver.get(url)

In [98]:
def get_url(search_term):
    """Generate url from search term"""
    template = "https://www.amazon.com/s?k={}&ref=nb_sb_noss_2"
    search_term = search_term.replace(' ', '+')
    return template.format(search_term)

In [99]:
url = get_url('Ring video doorbell')
print(url)

https://www.amazon.com/s?k=Ring+video+doorbell&ref=nb_sb_noss_2


In [12]:
driver.get(url)

Extracting the data

In [13]:
soup = BeautifulSoup(driver.page_source, "html.parser")

In [54]:
results = soup.find_all('div', {'data-component-type' : 's-search-result'})

In [55]:
len(results)

22

Prototype the record

In [56]:
item = results[0]

In [58]:
atag = item.h2.a

In [63]:
description = atag.text.strip()

In [80]:
url = 'https://www.amazom.com' + atag.get('href')

In [67]:
price_parent = item.find('span', 'a-price')

In [70]:
price = price_parent.find('span', 'a-offscreen').text

In [72]:
rating = item.i.text

'4.7 out of 5 stars'

In [79]:
#item.find('span', {'class' : 'a-size-base', 'dir' : 'auto'})
review_count = item.find('span', {'class' : 'a-size-base'}).text

Generalize the pattern

In [84]:
def extract_record(item):
    """Extract and return data from a single record"""
    #description and url
    atag = item.h2.a
    description = atag.text.strip()
    url = 'https://www.amazom.com' + atag.get('href')
    #price
    price_parent = item.find('span', 'a-price')
    price = price_parent.find('span', 'a-offscreen').text
    
    #ranking and rating
    rating = item.i.text
    review_count = item.find('span', {'class' : 'a-size-base'}).text
    
    result = (description, price, rating, review_count, url)
    return result
    
    

In [94]:
records =[]
results = soup.find_all('div', {'data-component-type' : 's-search-result'})

for item in results:
    record = extract_record(item)
    if record:
        records.append(record)

In [95]:
records[0]

('Ring Video Doorbell – newest generation, 2020 release – 1080p HD video, improved motion detection, easy installation – Satin Nickel',
 '$99.99',
 '4.7 out of 5 stars',
 '67,643',
 'https://www.amazom.com/Ring-Video-Doorbell-Satin-Nickel-2020-Release/dp/B08N5NQ869/ref=sr_1_1?dchild=1&keywords=Ring+video+doorbell&qid=1635688480&sr=8-1')

In [96]:
for row in records:
    print(row[1])

$99.99
$59.99
$119.99
$199.99
$99.99
$179.99
$99.99
$14.99
$169.99
$119.99
$149.99
$9.99
$79.99
$249.99
$84.99
$49.99
$49.99
$19.99
$19.99
$9.99
$14.99
$79.99


Getting the next page

In [100]:
def get_url(search_term):
    """Generate url from search term"""
    template = "https://www.amazon.com/s?k={}&ref=nb_sb_noss_2"
    search_term = search_term.replace(' ', '+')
    
    
    #add term query to url
    
    url = template.format(search_term)
    
    
    #add page query place holder
    url += "&page{}"
    
    
    return url

Combining all the codes

In [105]:
import csv
from bs4 import BeautifulSoup
from selenium import webdriver

def get_url(search_term):
    """Generate url from search term"""
    template = "https://www.amazon.com/s?k={}&ref=nb_sb_noss_2"
    search_term = search_term.replace(' ', '+')
    
    
    #add term query to url
    
    url = template.format(search_term)
    
    
    #add page query place holder
    url += "&page{}"
    
    
    return url

def extract_record(item):
    """Extract and return data from a single record"""
    #description and url
    atag = item.h2.a
    description = atag.text.strip()
    url = 'https://www.amazom.com' + atag.get('href')
    
    try:
        #price
        price_parent = item.find('span', 'a-price')
        price = price_parent.find('span', 'a-offscreen').text
    except AttributeError:
        return
    
    
    try:
        #ranking and rating
        rating = item.i.text
        review_count = item.find('span', {'class' : 'a-size-base'}).text
    except AttributeError:
        rating = ''
        review_count = ''
            
        
    
    result = (description, price, rating, review_count, url)
    return result

def main(search_term):
    """Run main program routine"""
    #startup the web driver
    driver = webdriver.Chrome()
    
    record =[]
    url = get_url(search_term)
    
    for page in range(1,21):
        driver.get(url.format(page))
        soup = BeautifulSoup(driver.page_source, "html.parser")
        results = soup.find_all('div', {'data-component-type' : 's-search-result'})
        
        for item in results:
            record = extract_record(item)
            if record:
                records.append(record)
                
    driver.close()
    
    #save data to csv file
    with open('results.csv', 'w', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['Description', 'Price', 'Rating', 'ReviewCount', 'Url'])
        writer.writerows(records)
        
    




In [106]:
main('Ring video doorbell')